In [32]:
import numpy as np
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
 
# Load training and test data into dataframes
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()


In [33]:
#TO DO: Convert y_train and y_test to categorical format
Y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)
 
# TODO: Split original training data to sub-training (80%) and validation data (20%)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.20)

In [34]:
# Each image's dimension is 28 x 28
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)
 
# Prepare the training images
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_train = X_train.astype('float32')
X_train /= 255
 
# TO DO: Prepare the test images as above
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
X_test = X_test.astype('float32')
X_test /= 255
 
# TO DO: Prepare the validation images as above
X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, 1)
X_val = X_val.astype('float32')
X_val /= 255

In [35]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
 
cnn3 = Sequential()

cnn3.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
cnn3.add(MaxPooling2D((2, 2)))
cnn3.add(Dropout(.2))

cnn3.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
cnn3.add(MaxPooling2D((2, 2)))
cnn3.add(Dropout(.2))

cnn3.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
cnn3.add(Dropout(.2))

cnn3.add(Flatten())
cnn3.add(Dense(128, activation='relu'))
cnn3.add(Dropout(.3))
cnn3.add(Dense(10, activation='softmax'))

cnn3.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_22 (Dropout)         (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 3, 3, 128)        

In [37]:
cnn3.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
 
#TO DO: Define ImageDataGenerator
gen = ImageDataGenerator(rescale=1./255)
batches = gen.flow(X_train, Y_train, batch_size=256)
val_batches = gen.flow(X_val, Y_val, batch_size=256)
 
history3 = cnn3.fit_generator(batches, steps_per_epoch=48000//256, epochs=50,
                    validation_data=val_batches, validation_steps=12000//256)
 

#TO DO: Evaluate results on the test set and print the results.
score3 =  cnn3.evaluate(X_test, Y_test)

Epoch 1/50
187/187 [==============================] - 14s 76ms/step - loss: 1.4409 - accuracy: 0.4641 - val_loss: 0.9029 - val_accuracy: 0.6943
Epoch 2/50
187/187 [==============================] - 14s 76ms/step - loss: 0.8586 - accuracy: 0.6788 - val_loss: 0.8683 - val_accuracy: 0.7164
Epoch 3/50
187/187 [==============================] - 14s 77ms/step - loss: 0.7852 - accuracy: 0.7012 - val_loss: 0.7127 - val_accuracy: 0.7439
Epoch 4/50
187/187 [==============================] - 14s 77ms/step - loss: 0.7386 - accuracy: 0.7204 - val_loss: 0.7127 - val_accuracy: 0.7527
Epoch 5/50
187/187 [==============================] - 15s 78ms/step - loss: 0.7078 - accuracy: 0.7309 - val_loss: 0.5939 - val_accuracy: 0.7569
Epoch 6/50
135/187 [====================>.........] - ETA: 3s - loss: 0.6870 - accuracy: 0.7368